In [ ]:
import sys
import os
from icecream import ic

sys.path.insert(0, "..")
sys.path.insert(0, "../../../Utilities")

sys.path.insert(0, "../../..")

import Ballpushing_utils
import Utils
import Processing
import HoloviewsTemplates

import importlib

import holoviews as hv

hv.extension("bokeh")

# Get the list of experiments

In [ ]:
# Get the data path
Datapath = Utils.get_data_path()

# Get all folders with "TNT_Fine" in the name

Folders = [
    f for f in os.listdir(Datapath) if "TNT_Fine" in f and "Tracked" in f and os.path.isdir(Datapath / f)
]

Folders

In [ ]:
importlib.reload(Ballpushing_utils)

In [ ]:
# Generate Experiment objects from each folder

Experiments = [Ballpushing_utils.Experiment(Datapath / f) for f in Folders]

In [ ]:
# Check some flies nicknames

TestFly = Experiments[15].flies[3].nickname

In [ ]:
TestFly

In [ ]:
savepath = Utils.get_labserver() / "Experimental_data/MultiMazeRecorder/Datasets/240227_TNT_Fine_Experiments.pkl"


In [ ]:
Ballpushing_utils.save_object(Experiments, savepath.as_posix())

In [ ]:
# Load the experiments from the saved file
Experiments = Ballpushing_utils.load_object(savepath.as_posix())

In [ ]:
type(Experiments)

In [ ]:
type(Experiments[0])

In [ ]:
importlib.reload(Ballpushing_utils)

In [ ]:
data = Ballpushing_utils.Dataset(Experiments)

In [ ]:
print(data)

In [ ]:
# For each fly in the dataset, if they have 2 nicknames, just keep the first one
# for fly in data.flies:
#     if len(fly.nickname) > 1:
#         fly.nickname = fly.nickname[0]

I used the above method as a hack to get rid of a supplementary nickname in PR flies. It is fixed directly in the brain region registry now and doesn't need to be used anymore.

In [ ]:
data.generate_dataset("summary")

In [ ]:
data.data["Nickname"]

In [ ]:
mydata = data.data

In [ ]:
noNa = mydata.dropna(subset=["FinalEvent"])

In [ ]:
# check for non numeric values in FinalEvent

noNa[noNa["FinalEvent"].apply(lambda x: not isinstance(x, int))]

In [ ]:
# Check for duplicate or missing indices

noNa[noNa.index.duplicated()]

In [ ]:
noNa = noNa.reset_index(drop=True)

In [ ]:
type(mydata["FinalEvent"][0])

In [ ]:
vdim = ["FinalEvent"]

In [ ]:
for metric in vdim:
    if metric in data.data.columns:
        print(f"{metric} exists in the dataset.")
    else:
        print(f"{metric} does not exist in the dataset.")

In [ ]:
# Assuming 'data' is your DataFrame and 'vdim' is your metric
print(f"Data type for {vdim}: {mydata[vdim].dtypes}")

In [ ]:
# Find unique data types I have in the FinalEvent column
noNa["FinalEvent"].apply(lambda x: type(x)).unique()

In [ ]:
data.jitter_boxplot(data.data, "NumberEvents", show=True, save=False)

In [ ]:
# Make a subset of the data that removes None rows from "FinalEvent"

subsetfinal = data.data.dropna(subset=["FinalEvent"])

In [ ]:
data.jitter_boxplot(subsetfinal, "FinalEvent", show=False, save=True)

# Troubleshooting the data error

I see that in some dataset loading I get: cannot reindex on an axis with duplicate labels
Current dataset. Let's check these.

First one is : 231222_TNT_Fine_1_Videos_Tracked_arena6_corridor5

In [ ]:
# Load the fly that is named "231222_TNT_Fine_1_Videos_Tracked_arena6_corridor5"

# Find which fly in the dataset has the name "231222_TNT_Fine_1_Videos_Tracked_arena6_corridor5"

flyname = "231222_TNT_Fine_1_Videos_Tracked_arena6_corridor5"

fly = [fly for fly in data.flies if fly.name == flyname][0]

In [ ]:
# Load the data from the fly
problematicFly = Ballpushing_utils.Dataset(fly)

In [ ]:
problematicFly.

In [ ]:
problematicFly.generate_dataset("summary")

In [ ]:
fly.arena_metadata

In [ ]:
problematicFly.flies[0].nickname

# PCA on the data

Here I'll try to do PCA on the data to see if I can get something interesting by reducing the dimensionality of the data, including all the summary metrics.

In [ ]:
# Subset the data to only include the label and metrics of interest
subset = data.data[
    [
        "NumberEvents",
        "FinalEvent",
        "FinalTime",
        "SignificantEvents",
        "SignificantFirst",
        "SignificantFirstTime",
        "CumulatedBreaks",
        "Pushes",
        "Pulls",
        "Genotype",
        "Brain region",
    ]
]

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import hvplot.pandas

# Separate the "TNTxZ2018" data from the rest of the data
TNTxZ2018_data = PCA_components[PCA_components["Genotype"] == "TNTxZ2018"]
other_data = PCA_components[PCA_components["Genotype"] != "TNTxZ2018"]

# Generate one plot per Brain region
for brain_region in PCA_components["Brain region"].unique():
    df_brain_region = other_data[other_data["Brain region"] == brain_region]

    # Create separate scatter plots for the "TNTxZ2018" genotype and the other genotypes
    plot1 = df_brain_region.hvplot.scatter(
        x="PC1", y="PC2", by="Genotype", hover_cols=["Genotype"], cmap="nipy_spectral"
    )
    plot2 = TNTxZ2018_data.hvplot.scatter(
        x="PC1",
        y="PC2",
        by="Genotype",
        hover_cols=["Genotype"],
        color="black",
        marker="x",
        size=100,
    )

    # Combine the plots
    final_plot = plot1 * plot2

    # Display the plot
    hvplot.show(final_plot)

In [ ]:
subset

In [ ]:
import hvplot.pandas

# Create a scatter plot for each brain region and genotype
plot = PCA_components.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="Genotype",
    groupby="Brain region",
    hover_cols=["Genotype"],
    cmap="nipy_spectral",
    dynamic=False,
)

# Adjust the appearance of the "TNTxZ2018" points
plot.opts(hv.opts.Scatter("TNTxZ2018", size=100, marker="x", color="black"))

# Display the plot
hvplot.show(plot)

In [ ]:
import hvplot.pandas
import holoviews as hv

# Separate the "TNTxZ2018" data from the rest of the data
TNTxZ2018_data = PCA_components[PCA_components["Genotype"] == "TNTxZ2018"]
other_data = PCA_components[PCA_components["Genotype"] != "TNTxZ2018"]

# Initialize an empty Layout
plots = hv.Layout()

# Generate one plot per Brain region
for brain_region in PCA_components["Brain region"].unique():
    df_brain_region = other_data[other_data["Brain region"] == brain_region]

    # Create separate scatter plots for the "TNTxZ2018" genotype and the other genotypes
    plot1 = df_brain_region.hvplot.scatter(
        x="PC1", y="PC2", by="Genotype", hover_cols=["Genotype"], cmap="nipy_spectral"
    )
    plot2 = TNTxZ2018_data.hvplot.scatter(
        x="PC1",
        y="PC2",
        by="Genotype",
        hover_cols=["Genotype"],
        color="black",
        marker="x",
        size=100,
    )

    # Combine the plots
    final_plot = plot1 * plot2

    # Add the plot to the Layout
    plots += final_plot.relabel(f"PCA - Brain Region: {brain_region}")

# Save the Layout
hvplot.save(plots.cols(1), "/mnt/labserver/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Plots/240222_PCA_plots.html")
# Display the Layout
hvplot.show(plots.cols(1))